In [ ]:
!pip install ortools

In [33]:
from ortools.linear_solver import pywraplp
from numpy import average, std
import json
import pandas as pd
import requests
import math

In [2]:
response1 = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json")
distances_stations_ntas_db = json.loads(response1.text)

In [4]:
with open("./functions/neighbors_with_9_minute_station.json", "r") as f:
    neighbors_with_9_minute_station = json.load(f)
neighbourhoods = list(neighbors_with_9_minute_station.keys())
K = len(neighbourhoods)
shifts = ["9 AM to 6 PM", "6 PM until 9 AM"]
I = len(shifts)
stations = list(distances_stations_ntas_db.keys())
L = len(stations)
vehicles = ["engine", "squad", "ladder", "hazardous", "rescue"]
J = len(vehicles)

In [10]:
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/estaciones.json")
stations_db = json.loads(response.text)
Cl = {}
for st in stations_db:
    Cl[stations.index(st["address"])] = st["capacity"]

In [45]:
def index2shift(i):
    return shifts[i]

def shift2index(shift):
    return shifts.index(shift)

def index2vehicle(j):
    return vehicles[j]

def vehicle2index(vehicle):
    return vehicles.index(vehicle)

def index2neighbourhood(k):
    return neighbourhoods[k]

def neighbourhood2index(neighbourhood):
    return neighbourhoods.index(neighbourhood)

def index2station(l):
    return stations[l]

def station2index(station):
    return stations.index(station)
    

In [35]:
Bijk = pd.read_csv("./aux_data/bijk.csv", index_col=0)
Bijk.columns = ["nta"] + vehicles + ["shift"]
newcols = []
for col in Bijk.columns[1:-1]:
    newcols.append(vehicle2index(col))
Bijk.columns = ["nta"] + newcols + ["shift"]
Bijk["nta"] = Bijk["nta"].apply(lambda x: neighbourhood2index(x))
Bijk = Bijk.apply(lambda x: x.apply(lambda y: math.ceil(y)))
Bijk.head()

,nta,0,1,2,3,4,shift
0,79,5,1,4,1,1,1
1,109,4,1,3,1,1,1
2,59,3,1,2,1,1,1
3,110,6,1,4,1,1,1
4,94,4,1,3,1,1,1


In [41]:
int(Bijk[(Bijk["nta"] == 79) & (Bijk["shift"] == 1)][0])

C:\Users\Hugo\AppData\Local\Temp\ipykernel_8900\1000147151.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  int(Bijk[(Bijk["nta"] == 79) & (Bijk["shift"] == 1)][0])


5

In [27]:
Skl = pd.read_csv("./aux_data/skl.csv")
newcols = []
for col in Skl.columns[1:]:
    newcols.append(neighbourhood2index(col))
Skl.columns = ["station"]+newcols
Skl["station"] = Skl["station"].apply(lambda x: station2index(x))
Skl.head()

,station,0,1,2,3,4,5,6,7,8,...,185,186,187,188,189,190,191,192,193,194
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [32]:
Pkv = pd.read_csv("./aux_data/pkv.csv", index_col=0)
boroughs = list(Pkv["boro_name"].unique())

def index2boro(l):
    return stations[l]

def boro2index(station):
    return stations.index(station)

Pkv["boro_name"] = Pkv["boro_name"].apply(lambda x: station2index(x))
Pkv.head()

array(['Queens', 'Bronx', 'Brooklyn', 'Manhattan', 'Staten Island'],
      dtype=object)

In [43]:
solver_scip = pywraplp.Solver.CreateSolver("SCIP") # Example, creating a SCIP solver
solver_cbc = pywraplp.Solver.CreateSolver("CBC") # Example, creating a CBC solver
solver_glop = pywraplp.Solver.CreateSolver("GLOP") # Example, creating a SCIP solver

In [47]:
solver = solver_glop

variables = {}

for i in range(I):
  for j in range(J):
    if j not in {1, 3}:
      for k in range(K):
        for l in range(L):
          v = solver.NumVar(0, solver.infinity(), "Vehicles of type " + index2vehicle(j) + " assigned to station " + index2station(l) + " to cover the neighbourhood " + index2neighbourhood(k) + " in shift " + index2shift(i))
          variables[(i,j,k,l)] = v
    else:
      for l in range(L):
        v = solver.BoolVar("Vehicles of type " + index2vehicle(j) + " assigned to station " + index2station(l) + " in shift " + index2shift(i))
        variables[(i,j,l)] = v

for l1 in range(L):
  for l2 in range(L):
    v = solver.NumVar(0, solver.infinity(), "Vehicles of type moved from station station " + index2station(l1) + " to station " + index2station(l2) + " in the change of shift")
    variables[(l1,l2)] = v

In [ ]:
objective = solver.Objective()
objective.SetMinimization()
for i in range(number_origins):
  for j in range(number_destinations):
    v = variables[(i,j)]
    cost = transportation_cost_per_unit[(i,j)]
    objective.SetCoefficient(v, cost)

In [ ]:
limit = []

for i in range(I):
  for j in range(J):
    for k in range(K):
      bijk = int(Bijk[(Bijk["nta"] == k) & (Bijk["shift"] == i)][j])
      c = solver.Constraint(-solver.infinity(), bijk)
      for l in range(L):
        if (i,j,k,l) in variables:
          v = variables[(i,j,k,l)]
          c.SetCoefficient(v, 1)
    limit.append(c)

In [ ]:
haz = []

for i in range(I):
  c = solver.Constraint(1, 1)
  for l in range(L):
    v = variables[(i,3,l)]
    c.SetCoefficient(v, 1)
  haz.append(c)


squads = []

for i in range(I):
  c = solver.Constraint(8, 8)
  for l in range(L):
    v = variables[(i,1,l)]
    c.SetCoefficient(v, 1)
  squads.append(c)

In [ ]:
vehiclej = []

for j in range(J):
  if j not in {1,3}:
    for i in range(I):
      c = solver.Constraint(0, T[j])
      for l in range(L):
        for k in range(K):
          v = variables[(i,j,k,l)]
          c.SetCoefficient(v, 1)
      vehiclej.append(c)

In [ ]:
nine_minutes = []

for k in range(K):
  for i in range(I):
    c = solver.Constraint(1, solver.infinity())
    for l in range(L):
      skl = Skl #FALTA POR SEGUIR DESDE AQUÍ, TENER EN CUENTA QUE TAMBIÉN HAY QUE CONTAR Yijl PARA VER SI ESTÁ DENTRO DE LOS 9 MIN
      is_inside = station_inside[(k,l)]
      for j in range(J):
        v = X[(i,j,k,l)]
        c.SetCoefficient(v, is_inside)
    nine_minutes.append(c)

In [ ]:
station_capacity = []

for l in range(L):
  capacity = Cl[l]
  for i in range(I):
    c = solver.Constraint(-solver.infinity(), capacity)
    for j in range(J):
      for k in range(K):
        v = X[(i,j,k,l)]
        c.SetCoefficient(v, 1)
    station_capacity.append(c)

In [ ]:
solver = pywraplp.Solver.CreateSolver("GLOP")

variables = {}

for i in range(number_origins):
  for j in range(number_destinations):
    v = solver.NumVar(0, solver.infinity(), "Units transported between: " + str((i,j)))
    variables[(i,j)] = v

objective = solver.Objective()
objective.SetMinimization()
for i in range(number_origins):
  for j in range(number_destinations):
    v = variables[(i,j)]
    cost = transportation_cost_per_unit[(i,j)]
    objective.SetCoefficient(v, cost)

stock_constraints = []

for i in range(number_origins):
  stock = stock_at_origin[i]
  c = solver.Constraint(-solver.infinity(), stock)
  for j in range(number_destinations):
    v = variables[(i,j)]
    c.SetCoefficient(v, 1)
  stock_constraints.append(c)

demand_constraints = []

for j in range(number_destinations):
  min_demand = demand_at_destination[j]
  c = solver.Constraint(min_demand, solver.infinity())
  for i in range(number_origins):
    v = variables[(i,j)]
    c.SetCoefficient(v,1)
  demand_constraints.append(c)

result = solver.Solve()

if result == solver.ABNORMAL :
  print("Execution finished by an error")
elif result == solver.FEASIBLE :
  print("In the specified time limit the solver has found a feasible solution")
  for i in range(number_origins):
    for j in range(number_destinations):
      v = variables[(i,j)]
      if v.SolutionValue()>0:
        print(v, v.solution_value())
  print("The value for the objective function is", objective.Value())
elif result == solver.INFEASIBLE :
  print("There is no feasible solution for the problem")
elif result == solver.NOT_SOLVED :
  print("In the specified time limit the solver has not found any feasible solution")
elif result == solver.OPTIMAL :
  print("In the specified time limit the solver has found a feasible solution")
  for i in range(number_origins):
    for j in range(number_destinations):
      v = variables[(i,j)]
      if v.SolutionValue()>0:
        print(v, v.solution_value())
  print("The optimal value for the objective function is", objective.Value())
elif result == solver.UNBOUNDED :
  print("The solution is unbounded")
else :
  print("Unknown error code")